In [2]:
import pandas as pd
import gurobipy as gp
import numpy as np
from gurobipy import Model, GRB
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

In [3]:
#Load data
warehouse_data = pd.read_excel("NLP.xlsx", sheet_name="Warehouse", header=None)

In [4]:
warehouse_data

,0,1,2,3,4
0,W-Mart Warehouse Location,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,Inputs,NaN,NaN,NaN,NaN
3,Retail Stores,"X-coord., miles","Y-coord., miles",Number of Annual Shipments,"Distance from Warehouse to Stores, miles"
4,Store 1,5,10,180,11.18034
5,Store 2,10,5,300,11.18034
6,Store 3,0,12,200,12
7,Store 4,12,2,400,12.165525
8,Store 5,8,11,290,13.601471
9,Store 6,4,6,160,7.211103


In [5]:
stores=np.array(warehouse_data.iloc[4:10, 0:5])

In [6]:
# Extract coordinates and shipments
x_coords, y_coords, shipments = stores[:,1], stores[:,2], stores[:,3]

In [7]:
shipments[2]

200

In [8]:
# Create Gurobi model
m = gp.Model("Warehouse_Location")

# Decision Variables: Warehouse coordinates (Continuous)
X = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="X_Coord")
Y = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Y_Coord")
z = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Y_Coord")

# Objective: Minimize weighted distance
distance_expr = gp.quicksum(shipments[i] * ((X - x_coords[i])**2 + (Y - y_coords[i])**2)**0.5 for i in range(len(stores)))
m.addConstr(z == distance_expr, "Aux variable")

m.setObjective(z, GRB.MINIMIZE)

# Allow non-convex optimization (due to sqrt)
m.Params.NonConvex = 2

# Optimize model
m.optimize()

# Print results
if m.status == GRB.OPTIMAL:
    print(f"Optimal Warehouse Location: ({X.X:.2f}, {Y.X:.2f})")
    print(f"Minimum Total Weighted Distance: {m.ObjVal:.2f}")

Set parameter Username
Set parameter LicenseID to value 2656916
Academic license - for non-commercial use only - expires 2026-04-25
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Non-default parameters:
NonConvex  2

Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0xe61be7c5
Model has 1 general nonlinear constraint (18 nonlinear terms)
Variable types: 3 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve model has 1 nlconstr
Added 14 variables to disaggregate expressions.
Presolve time: 0.01s
Presolved: 43 rows, 18 columns, 100 nonzeros
Presolved model has 2 bilinear constraint(s)
Presolved 

In [12]:
# Create Gurobi model (changed shiopments for Store 3 to 500
shipments[2]=200
shipments[3]=1000
m = gp.Model("Warehouse_Location")

# Decision Variables: Warehouse coordinates (Continuous)
X = m.addVar(lb=10, vtype=GRB.CONTINUOUS, name="X_Coord")
Y = m.addVar(lb=2, vtype=GRB.CONTINUOUS, name="Y_Coord")
z = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Y_Coord")

# Objective: Minimize weighted distance
distance_expr = gp.quicksum(shipments[i] * ((X - x_coords[i])**2 + (Y - y_coords[i])**2)**0.5 for i in range(len(stores)))
m.addConstr(z == distance_expr, "Aux variable")
m.addConstr(Y >= 2)
m.addConstr(X >= 2)

m.setObjective(z, GRB.MINIMIZE)

# Allow non-convex optimization (due to sqrt)
m.Params.NonConvex = 2

# Optimize model
m.optimize()

# Print results
if m.status == GRB.OPTIMAL:
    print(f"Optimal Warehouse Location: ({X.X:.2f}, {Y.X:.2f})")
    print(f"Minimum Total Weighted Distance: {m.ObjVal:.2f}")

Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Non-default parameters:
NonConvex  2

Optimize a model with 2 rows, 3 columns and 2 nonzeros
Model fingerprint: 0x5a4e8104
Model has 1 general nonlinear constraint (18 nonlinear terms)
Variable types: 3 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+00, 1e+01]
  RHS range        [2e+00, 2e+00]
Presolve model has 1 nlconstr
Added 14 variables to disaggregate expressions.
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 43 rows, 18 columns, 102 nonzeros
Presolved model has 2 bilinear constraint(s)
Presolved model has 6 nonlinear constraint(s)

Solving non-convex MINLP

Variable types: 18 continuous, 

In [8]:
storesIndexed = {warehouse_data.iloc[i+4, 0]: warehouse_data.iloc[i+4, 1:4].tolist() for i in range(6)} 
storesIndexed

{'Store 1': [5, 10, 180],
 'Store 2': [10, 5, 300],
 'Store 3': [0, 12, 200],
 'Store 4': [12, 2, 400],
 'Store 5': [8, 11, 290],
 'Store 6': [4, 6, 160]}

In [9]:
# Create the Gurobi model
model = gp.Model("Warehouse_location")

# Decision Variables: Warehouse coordinates (Continuous)
X = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="X_Coord")
Y = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Y_Coord")

# Objective: Minimize weighted distance
squared_distance = (X - x_coords)**2 + (Y - y_coords)**2
customers = model.addVars(storesIndexed.keys())
model.addConstrs(
    customers[customer] == (X - storesIndexed[customer][0])**2 + (Y - storesIndexed[customer][1])**2
    for customer in storesIndexed.keys()
)
customer_distances = model.addVars(storesIndexed.keys())
for customer in storesIndexed.keys():
    model.addGenConstrPow(customers[customer], customer_distances[customer], .5, customer, "FuncPieceError=0.000001")

model.setObjective(
    gp.quicksum(customer_distances[customer] * storesIndexed[customer][2]
                for customer in storesIndexed.keys()),
    GRB.MINIMIZE
)


# Allow non-convex optimization (due to sqrt)
model.Params.NonConvex = 2

# Optimize model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Warehouse Location: ({X.X:.2f}, {Y.X:.2f})")
    print(f"Minimum Total Weighted Distance: {model.ObjVal:.2f}")
    

Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Non-default parameters:
NonConvex  2

Optimize a model with 0 rows, 14 columns and 0 nonzeros
Model fingerprint: 0x3443a256
Model has 6 quadratic constraints
Model has 6 function constraints treated as nonlinear
  6 POW
Variable types: 14 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+01]
  Objective range  [2e+02, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [5e+01, 2e+02]
Presolve time: 0.00s
Presolved: 42 rows, 17 columns, 93 nonzeros
Presolved model has 2 bilinear constraint(s)
Presolved model has 6 nonlinear constraint(s)
         in nonlinear terms.
        